In [1]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=912d5ee6eed46a5e3db653eb862c4de44ed2a1883c873e72dd81d9fccbe3f522
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


In [2]:
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import requests

import numpy as np
import pandas as pd

In [3]:
ua = UserAgent()
header = {'user-agent': ua.chrome} 

In [48]:
def scraping_film_data(page):
    link = 'https://mydramalist.com'
    film_link = 'https://mydramalist.com/search?adv=titles&ty=68&co=3&re=1980,2021&rt=1,10&so=alpha&page='
    film_resp = requests.get(film_link+page, headers=header)
    if film_resp.status_code == 200:
      film_html = film_resp.text
      film_soup = BeautifulSoup(film_html, 'lxml')
      page = film_soup.find('div', {'class':'m-t nav-active-border b-primary'})
      box_tag = page.find_all('div', {'class':'box'})
      film_data = []
      for box in box_tag:
          title_tag = box.find('h6', {'class':'text-primary title'})
          if title_tag is not None:
            title_link = title_tag.find('a').get('href')
            title_page = requests.get(link+title_link, headers=header)

            if title_page.status_code == 200:
              title_html = title_page.text
              title_soup =  BeautifulSoup(title_html, 'lxml')
              detail_tag = title_soup.find_all('div', {'class':'box clear hidden-sm-down'})
              detail_info = []
              for detail in detail_tag:
                list_info = detail.find_all('li', {'class':'list-item p-a-0'})                
                if list_info is not None:
                  title_info = [info.text.strip() for info in list_info]
                  detail_info.extend(title_info)

              del detail_info[-1]
              if len(detail_info) != 11:
                if not detail_info[4].startswith('Aired On'):
                  detail_info.insert(4, 'Aired On:N/A')
                if not detail_info[5].startswith('Original'):
                  detail_info.insert(5, 'Original Network:N/A')
                if not detail_info[6].startswith('Duration'):
                  detail_info.insert(6, 'Duration:N/A') 
              
              content_info = title_soup.find('li', {'class':'list-item p-a-0 content-rating'})
              if content_info is not None:
                content = content_info.text.strip()
              else:
                content = 'Content Rating:N/A'
          
              genres_info = title_soup.find('li', {'class':'list-item p-a-0 show-genres'})
              if genres_info is not None:
                genre = genres_info.text.strip() 
              else:
                genre = 'Genres:N/A'
                  
              tags_info = title_soup.find('li', {'class':'list-item p-a-0 show-tags'})
              if tags_info is not None:
                tag = tags_info.text.strip()
                tag = tag.replace('(Vote or add tags)','').strip()
              else:
                tag = 'Tags:N/A'   

              detail_link = link + title_link
              detail_link = detail_link.replace('https://', 'Link:')
              detail_info.extend([content, genre, tag, detail_link])
              film_info = [x.split(':')[1].strip() if x.split(':')[1].strip() != '' else 'N/A' for x in detail_info]
              film_data.append(film_info)

    return film_data

In [50]:
get_film = []
for p in range(1, 134):
    film = scraping_film_data(str(p))
    df = pd.DataFrame(film)
    df.columns = ['Title','Country','Episodes','Aired','Aired On','Original Network','Duration',
                  'Score','Ranked','Popularity','Watchers','Content Rating','Genres','Tags','Link']
    get_film.append(df)

df_film = pd.concat(get_film).reset_index(drop=True)
df_film

,Title,Country,Episodes,Aired,Aired On,Original Network,Duration,Score,Ranked,Popularity,Watchers,Content Rating,Genres,Tags,Link
0,1% of Anything,South Korea,26,"Jul 6, 2003 - Dec 28, 2003",Sunday,MBC,52 min.,"6.9 (scored by 2,242 users)",#4331,#1272,"4,829",13+ - Teens 13 or older,"Comedy, Romance","Arranged Relationship, Love/Hate Relationship,...",mydramalist.com/198-1-of-anything
1,100 Days My Prince,South Korea,16,"Sep 10, 2018 - Oct 30, 2018","Tuesday, Monday",tvN,1 hr. 15 min.,"8.3 (scored by 11,767 users)",#348,#161,"25,853",15+ - Teens 15 or older,"Historical, Comedy, Romance, Drama","Amnesia, Hidden Identity, Prince Lead, Joseon ...",mydramalist.com/27392-hundred-days-husband
2,100% Era,South Korea,8,"Jan 7, 2021 - Feb 25, 2021",Thursday,N/A,15 min.,7.6 (scored by 50 users),#9956,#8062,356,15+ - Teens 15 or older,"Friendship, School, Youth","COVID-19, Teenager, Rivalry, Steamy Kiss, Mini...",mydramalist.com/685329-100-era
3,101st Proposal,South Korea,15,"May 29, 2006 - Jul 25, 2006","Monday, Tuesday",SBS,1 hr. 5 min.,7.1 (scored by 119 users),#17184,#7135,441,15+ - Teens 15 or older,Romance,N/A,mydramalist.com/776-101st-proposal
4,109 Strange Things,South Korea,6,"Feb 1, 2017 - Feb 16, 2017","Wednesday, Thursday",Naver TV Cast,13 min.,6.8 (scored by 904 users),#4480,#2362,"2,422",Not Yet Rated,"Romance, Youth, Sci-Fi","College Student, Miniseries, Interspecies Roma...",mydramalist.com/21779-109-strange-things
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,Youth Syndrome,South Korea,10,"Aug 24, 2016 - Oct 27, 2016",N/A,N/A,N/A,7.7 (scored by 4 users),#9835,#99999,14,Not Yet Rated,"Comedy, Romance",Web Series,mydramalist.com/63649-youth-syndrome
2656,Youtuber Class,South Korea,4,"Dec 27, 2020 - Jan 17, 2021",Sunday,"Naver TV Cast, vLive",11 min.,7.7 (scored by 69 users),#9341,#8382,328,Not Yet Rated,"School, Youth","YouTuber, Cherry Bullet, DONGKIZ, Dream, Teach...",mydramalist.com/683959-youtuber-class
2657,"Yum Yum, How Tasty!",South Korea,4,"May 5, 2019 - May 24, 2019",N/A,N/A,7 min.,7.5 (scored by 2 users),#99999,#99999,8,Not Yet Rated,N/A,"Office Worker, Miniseries, Web Series",mydramalist.com/65629-yum-yum-how-tasty
2658,Zombie Detective,South Korea,24,"Sep 21, 2020 - Oct 27, 2020","Monday, Tuesday",KBS2,30 min.,"8.2 (scored by 3,026 users)",#507,#715,"8,539",15+ - Teens 15 or older,"Comedy, Drama, Zombies, Fantasy","Detective, Investigation, Mystery, Strong Fema...",mydramalist.com/61063-zombie-detective


In [51]:
df_film.to_csv('drama_korea_data.csv', index=False)